## Get all historical token rebalance data

In [29]:
%load_ext lab_black

import requests
import pandas as pd
import numpy as np
import tqdm
import pickle

from datetime import datetime, timedelta, date

pd.set_option("max_columns", 99)
pd.options.display.float_format = "{:.12f}".format

In [2]:
token_data =pd.DataFrame(requests.get(f'https://ftx.com/api/lt/tokens').json()["result"])
token_data = token_data[token_data.currentLeverage!= 0]

In [3]:
token_data[token_data["name"].str.contains("COMP")]

,name,description,underlying,leverage,outstanding,pricePerShare,positionPerShare,positionsPerShare,basket,targetComponents,underlyingMark,totalNav,totalCollateral,contractAddress,bep2AssetName,currentLeverage,change1h,change24h,changeBod,greeks
60,COMPBULL,3X Long Compound Token Token,COMP-PERP,3.0,9.960827e+06,0.010141,2.309732e-04,{'COMP-PERP': 0.00023097323533372013},"{'COMP-PERP': 0.00023097323533372013, 'USD': -...",[COMP-PERP],141.93,101013.247200,101059.260890,None,None,3.232607,-0.010573,-0.049643,-0.092774,NaN
61,COMPHALF,0.5X Long Compound Token Token,COMP-PERP,0.5,8.043341e-01,10870.524166,3.765748e+01,{'COMP-PERP': 37.65748445860429},"{'COMP-PERP': 37.65748445860429, 'USD': 5525.7...",[COMP-PERP],141.93,8743.533598,8744.139382,None,None,0.491671,-0.001623,-0.007251,-0.015258,NaN
62,COMPHEDGE,1X Short Compound Token Token,COMP-PERP,-1.0,7.342488e+01,79.585312,-5.258613e-01,{'COMP-PERP': -0.5258612762364566},"{'COMP-PERP': -0.5258612762364566, 'USD': 154....",[COMP-PERP],141.93,5843.541796,5842.769570,None,None,-0.937805,0.003110,0.012770,0.028983,NaN
63,COMPBEAR,3X Short Compound Token Token,COMP-PERP,-3.0,9.248338e+09,0.000002,-3.223288e-08,{'COMP-PERP': -3.2232882257982886e-08},"{'COMP-PERP': -3.2232882257982886e-08, 'USD': ...",[COMP-PERP],141.93,15981.567039,15975.605027,None,None,-2.647389,0.007880,0.034860,0.091025,NaN


In [4]:
token_data.shape

(182, 20)

### All History

In [ ]:
all_data = {}
df_all_data  =  pd.DataFrame()

for market in tqdm.tqdm(token_data[['name', 'underlying']].values):
    market_data = pd.DataFrame()
    
    current_date = int(datetime.now().timestamp())
    max_date = date(2020,1,1)

    while True:
        # End if no data returned
        try:
            df = pd.DataFrame(requests.get(f'https://ftx.com/api/lt/{market[0]}/major_rebalances?end_time={current_date}').json()["result"])
        except:
            break
        
        market_data = market_data.append(df, ignore_index=False)
        
        # End if <100 results
        if df.shape[0] < 100:
            break
        
        # End if gone back far enough
        if pd.to_datetime(df.time).iloc[0] < max_date:
            break

        current_date = pd.to_datetime(df.time).astype(int).iloc[-1] / 10**9
    
    market_data.drop_duplicates(inplace=True)
    market_data['time'] = pd.to_datetime(market_data['time'])
    market_data['market'] = market[0]
    market_data['underlying'] = market[1]
    
    
    all_data[market[0]] = market_data
    
    df_all_data = df_all_data.append(market_data, ignore_index=True)
    market_data.to_pickle(f"rebalance_data/{market[0]}")

### Recent History Only

In [5]:
all_data = {}
df_all_data  =  pd.DataFrame()

for market in tqdm.tqdm(token_data[['name', 'underlying']].values):
    market_data = pd.DataFrame()
    
    current_date = int(datetime.now().timestamp())
    max_date = date(2020,1,1)
    
    df = pd.DataFrame(requests.get(f'https://ftx.com/api/lt/{market[0]}/major_rebalances').json()["result"])
    market_data = market_data.append(df, ignore_index=False)
    
    market_data.drop_duplicates(inplace=True)
    market_data['time'] = pd.to_datetime(market_data['time'])
    market_data['market'] = market[0]
    market_data['underlying'] = market[1]
    
    
    all_data[market[0]] = market_data
    
    df_all_data = df_all_data.append(market_data, ignore_index=True)

100%|██████████| 182/182 [01:48<00:00,  1.68it/s]


In [11]:
from datetime import  time
df_all_data = df_all_data.loc[
            (df_all_data["time"].dt.time >= time(0, 2, 0))
            & (df_all_data["time"].dt.time <= time(0, 3, 30))
        ].sort_values("time")
df_all_data["date"]  =  df_all_data.time.dt.date

In [17]:
df_all_data["filledSize"] = np.where(df_all_data["side"] =="sell", df_all_data["filledSize"]*-1,  df_all_data["filledSize"])

In [33]:
latest_d = df_all_data["date"].max()
df_latest = (
    df_all_data[df_all_data["date"] == latest_d]
    .groupby(["date", "underlying"])
    .usd_notional.sum()
    .reset_index()
)

In [35]:
display(df_latest.sort_values("usd_notional").head())
display(df_latest.sort_values("usd_notional").tail())

,date,underlying,usd_notional
44,2022-04-21,XRP-PERP,-1319372.586249999935
18,2022-04-21,ETH-PERP,-820792.489400000079
6,2022-04-21,BCH-PERP,-697477.445099999895
25,2022-04-21,LTC-PERP,-484794.794799999916
43,2022-04-21,XLM-PERP,-308766.816325000022


,date,underlying,usd_notional
11,2022-04-21,COMP-PERP,21045.691855000001
36,2022-04-21,TOMO-PERP,25182.998785000003
33,2022-04-21,SUSHI-PERP,28560.641150000003
22,2022-04-21,KNC-PERP,31399.437549999995
16,2022-04-21,EOS-PERP,712919.544774999842
